Imports

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.crs as ccrs

from hlpr_func import make_cyclic, sindeg, lat_mean, calc_trend_HAC, add_weights, get_weights_values

nicename = {"F_TOA": "$F^\\uparrow_{TOA}$",
             "F_surf": "$F_{surf}$",
             "F_clr": "$F_{clr}$",
             "F_cloud": "$F_{cloud}$",
             "aod": "AOD",
             "ssaod550": "sea salt AOD",
             "si10": "wind speed"}
plotcolor = {"F_TOA": "black",
             "F_surf": "mediumseagreen",
             "F_clr": "hotpink",
             "F_cloud": "deepskyblue",
             "aod": "brown",
             "ssaod550": "darkblue",
             "si10": "black"}

units = {"F_TOA":"W m$^{-2}$",
         "F_clr":"W m$^{-2}$",
         "aod":"",
         "ssaod550":"",
         "si10":"m s$^{-1}$",}

Load data

In [ ]:
ceres = make_cyclic(xr.open_mfdataset("CERES-data/CERES_Cru23_ds.nc"))

terra = xr.open_dataset("MODIS_AOD/Terra_combined.nc").sortby("lat")
terra["lon"] = (terra.coords['lon'] + 360) % 360
terra = make_cyclic(terra.sortby("lon").sel(time=slice("2000-03","2023-12")))
terra = terra.rename_vars({"aod":"aod_terra", "sza":"sza_terra"})
ds = xr.merge([ceres, terra])
ceres.close()
terra.close()

aqua = xr.open_dataset("MODIS_AOD/Aqua_combined.nc").sortby("lat")
aqua["lon"] = (aqua.coords['lon'] + 360) % 360
aqua = make_cyclic(aqua.sortby("lon").sel(time=slice("2000-03","2023-12")))
aqua = aqua.rename_vars({"aod":"aod_aqua", "sza":"sza_aqua"})
ds = xr.merge([ds, aqua])
aqua.close()

era = xr.open_dataset("ERA5-CAMS/era5-interpolated.nc")
era["lsmask"] = (era.sst / era.sst).fillna(0)
ds = xr.merge([ds, era])
era.close()

aod_modis = np.nanmean(np.stack([ds.aod_aqua.values, ds.aod_terra.values]), axis=0)
aod_modis = xr.DataArray(aod_modis, coords=ds.aod_aqua.coords, dims=ds.aod_aqua.dims)
ds["aod"] = aod_modis

ds = ds[["F_surf","F_clr","F_cloud","F_TOA","aod","landmask","icemask","limask"]]

ds

In [ ]:
da = ds.assign_coords({"month_since_start": ("time", np.arange(len(ds.time.values)))})

# add weights
weights = da.time.dt.days_in_month
weights = weights.where(weights.time.dt.month!=2, 28.65)
da["days_in_month"] = weights

climatology = da.groupby("time.month").mean("time")
deseasonalized = da.groupby("time.month") - climatology + climatology.weighted(climatology.days_in_month).mean("month")
so = lat_mean(deseasonalized.sel(lat=slice(-75,-45)).where(deseasonalized.landmask==0).mean("lon"))

djf = da.where(da.time.dt.month.isin([12,1,2]))
# remove austrailian bush fires (Dec 2019 - Mar 2020)
djf = djf.where(~(djf.time.dt.year.isin(2019) & djf.time.dt.month.isin(12)))
djf = djf.where(~(djf.time.dt.year.isin(2020) & djf.time.dt.month.isin([1,2])))
djf["days_in_month"] = weights

climatology = djf.groupby("time.month").mean("time")
deseasonalized = djf.groupby("time.month") - climatology + climatology.weighted(climatology.days_in_month).mean("month")
so_djf = lat_mean(deseasonalized.sel(lat=slice(-75,-45)).where(deseasonalized.landmask==0).mean("lon"))
so_djf["days_in_month"] = weights
so_djf

In [ ]:
da = ds.sel(time=slice("2003","2023"))[["F_TOA","F_clr","F_cloud","F_surf","aod"]]
da = add_weights(da)
da = da.assign_coords({"month_since_2003-01": ("time", np.arange(len(da.time.values)))})
da = da.swap_dims({"time":"month_since_2003-01"})
fit = da.polyfit("month_since_2003-01", 1, w=da.days_in_month).drop_vars("days_in_month_polyfit_coefficients")
da.close()

fit

In [ ]:
fig = plt.figure(figsize=(10,5))

crange = {"F_clr":3, "aod":0.15}
for i,var in enumerate(["F_clr","aod"]):
    ax = fig.add_subplot(2,2,i*2+1,projection=ccrs.Robinson())
    
    trend = fit.sel(degree=1)[var+"_polyfit_coefficients"] * 120 # convert from per month to per decade

    cf = ax.pcolormesh(fit.lon, fit.lat, trend, vmin=-crange[var], vmax=crange[var],
                cmap="bwr", transform=ccrs.PlateCarree())

    ax.text(x=-0.1,y=0.5, s=nicename[var],fontsize=18,
        rotation=90,horizontalalignment='center',verticalalignment='center',transform=ax.transAxes)
    ax.coastlines()
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.5, color='gray', linestyle=':')
    cb = plt.colorbar(cf, extend="both", orientation='vertical', shrink=0.7, ticks = np.linspace(-crange[var],crange[var],3))
    if units[var] != "":
        cb.set_label(label="Trend ["+units[var]+" decade$^{-1}$]", size=9)
    else:
        cb.set_label(label="Trend [decade$^{-1}$]", size=9)
    
    rect = mpatches.Rectangle(xy=[-75, -180], width=360, height=30,
                              facecolor=None, edgecolor="k",
                              transform=ccrs.PlateCarree())
    ax.add_patch(rect)

    ##
    
    ax = fig.add_subplot(2,2,i*2+2)
    y = so[var].drop_sel(time="2004-10-15").dropna(dim="time")
    t = y.month_since_start.values
    w = get_weights_values(y)
    fitted, trend, trend_pm = calc_trend_HAC(t, y.values, w, hac_maxlags=12)

    if i == 0:
        ax.plot(y.time, y, color=plotcolor[var], label=f"{trend*120:.2f} $\\pm$ {trend_pm*120:.2f} "+units[var]+" decade$^{-1}$")
    else:
        ax.plot(y.time, y, color=plotcolor[var], label=f"{trend*120:.3f} $\\pm$ {trend_pm*120:.3f}"+" decade$^{-1}$")
    ax.plot(y.time, fitted, color=plotcolor[var], lw=0.5)
    ax.legend(loc=2, frameon=False)

    if i == 0:
        ax1 = ax
        ax.set_title("Southern Ocean mean")
    else:
        ax.sharex(ax1)

plt.savefig("talk-figures/Fclr_AOD_trends.png", dpi=200, facecolor="w", bbox_inches="tight")
plt.show()

In [ ]:
dt = xr.open_dataset("CERES-data/trends_Cru23_N50.nc")
dt

In [ ]:
fig = plt.figure(figsize=(10,9), constrained_layout=True)
gs = fig.add_gridspec(ncols=2, nrows=3, width_ratios=[1.2, 1], height_ratios=[1,1.1,1.1], wspace=0.01)

crange = {"F_clr":3, "aod":0.15}
for i,var in enumerate(["F_clr","aod"]):

    # add map
    ax = fig.add_subplot(gs[i+1,0],projection=ccrs.Robinson())
    
    trend = fit.sel(degree=1)[var+"_polyfit_coefficients"] * 120 # convert from per month to per decade

    cf = ax.pcolormesh(fit.lon, fit.lat, trend, vmin=-crange[var], vmax=crange[var],
                cmap="bwr", transform=ccrs.PlateCarree())

    ax.text(x=-0.08,y=0.5, s=nicename[var],fontsize=18,
        rotation=90,horizontalalignment='center',verticalalignment='center',transform=ax.transAxes)
    ax.coastlines()
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.5, color='gray', linestyle=':')

    cax = ax.inset_axes([1.02, 0.05, 0.025, 0.9])  # [left, bottom, width, height] in axes fraction
    cb = fig.colorbar(cf, cax=cax, extend="both", orientation='vertical', ticks=np.linspace(-crange[var], crange[var], 3))
    if units[var] != "":
        cb.set_label(label="Trend ["+units[var]+" decade$^{-1}$]", size=9)
    else:
        cb.set_label(label="Trend [decade$^{-1}$]", size=9) 
    
    rect = mpatches.Rectangle(xy=[-178,-75], width=356, height=30,
                              facecolor="none", edgecolor="k", linewidth=2,
                              transform=ccrs.PlateCarree())
    ax.add_patch(rect)
    if i == 0:
        ax.set_title("b)", loc='left')
    else:
        ax.set_title("c)", loc='left')

    # add SO timeseries
    ax = fig.add_subplot(gs[i+1,1])
    y = so[var].drop_sel(time="2004-10-15").dropna(dim="time")
    t = y.month_since_start.values
    w = get_weights_values(y)
    fitted, trend, trend_pm = calc_trend_HAC(t, y.values, w, hac_maxlags=12)

    if i == 0:
        ax.plot(y.time, y, color=plotcolor[var], label=f"{trend*120:.2f} $\\pm$ {trend_pm*120:.2f} "+units[var]+" decade$^{-1}$")
    else:
        ax.plot(y.time, y, color=plotcolor[var], label=f"{trend*120:.3f} $\\pm$ {trend_pm*120:.3f}"+" decade$^{-1}$")
    ax.plot(y.time, fitted, color=plotcolor[var], lw=0.5)
    ax.legend(loc=2, frameon=False)

    if i == 0:
        ax1 = ax
        ax.set_title("Southern Ocean mean")
        ax.set_title("d)", loc='left')
    else:
        ax.sharex(ax1)
        ax.set_title("e)", loc='left')

# add zm trend in all CERES components
ax = fig.add_subplot(gs[0,:])
for i,var in enumerate(["F_surf","F_clr","F_cloud","F_TOA"]):
    nh = dt.sel(comp="NH").sel(rad=var)
    sh = dt.sel(comp="SH").sel(rad=var)
    sh["lat"] = sh.lat * -1
    da = xr.concat([sh,nh], dim="lat")
    da = da.sortby("lat")
    
    ax.plot(da.lat, da.signif_trend * 120, lw=2, color=plotcolor[var], label=nicename[var])
    ax.plot(da.lat, da.trend * 120, lw=1, ls=":", color=plotcolor[var], label="not significant" if var == "F_TOA" else "")
    ax.fill_between(da.lat, 0, da.trend * 120, color=plotcolor[var], alpha=0.1 if plotcolor[var] == "k" else 0.3)
    ax.set_ylabel("Trend [W m$^{-2}$ decade$^{-1}$]")
    ax.set_ylim(-2.1,1.1)
    ax.legend(frameon=False, loc=3, ncols=2)
    xticks = [-90, -45, -30, -15, 0, 15, 30, 45, 90]
    ax.set_xticks(sindeg(xticks),[str(xt)+"°" for xt in xticks])
    ax.set_xlim(-1,1)
    ax.set_title("a)", loc='left')

plt.savefig("paper-figures/FTOA_AOD_trends.png", dpi=200, facecolor="w", bbox_inches="tight")
plt.show()

In [ ]:
fig,axes = plt.subplots(2, 4, figsize=(12,5), subplot_kw={"projection":ccrs.Robinson()}, constrained_layout=True)
letter = ["a)", "b)", "c)", "d)", "e)", "f)", "g)", "h)"]

for i,var in enumerate(["F_TOA","F_cloud","F_surf","F_clr"]):
    ax = axes[0,i]
    clim = ds[var].mean("time")
    cf1 = ax.pcolormesh(clim.lon, clim.lat, clim, vmin=25, vmax=150,
                cmap="Blues", transform=ccrs.PlateCarree())
    ax.set_title(nicename[var])
    ax.set_title(letter[i], loc="left")
    ax.coastlines()
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.5, color='gray', linestyle=':')


    ax = axes[1,i]
    trend = fit.sel(degree=1)[var+"_polyfit_coefficients"] * 120 # convert from per month to per decade
    cf2 = ax.pcolormesh(fit.lon, fit.lat, trend, vmin=-8, vmax=8,
                cmap="bwr", transform=ccrs.PlateCarree())
    ax.coastlines()
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, linewidth=0.5, color='gray', linestyle=':')
    ax.set_title(letter[i+4], loc="left")


cbar_ax = fig.add_axes([0.35, 0.54, 0.3, 0.02])
cbar=fig.colorbar(cf1, cax=cbar_ax, extend="both", ticks=np.linspace(25,150,6),
                  orientation='horizontal', label="Climatology [W m$^{-2}$]")

cbar_ax = fig.add_axes([0.35, 0.04, 0.3, 0.02])
cbar=fig.colorbar(cf2, cax=cbar_ax, extend="both", ticks=np.linspace(-8,8,5),
                  orientation='horizontal', label="Trend [W m$^{-2}$ decade$^{-1}$]")

plt.savefig("paper-figures/CERES_clim_trends.png", dpi=200, facecolor="w", bbox_inches="tight")
plt.show()